<a href="https://colab.research.google.com/github/copy-del/my-repo/blob/main/NLP%EB%AA%A8%EB%8D%B8_%ED%99%9C%EC%9A%A9_%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 마운트
import google.colab.drive
google.colab.drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/실전프로젝트

/content/drive/MyDrive/Colab Notebooks/실전프로젝트


In [3]:
# dart api install
!pip install dart_fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 3.1.0
    Uninstalling termcolor-3.1.0:
      Successfully uninstalled termcolor-3.1.0


In [12]:
# 필요한 라이브러리 import
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import zipfile
import io
from bs4 import BeautifulSoup

In [7]:
# NLP 모델을 활용해 DART API에서 추출한 3개년 사업보고서를 요약하자
# 필요한 라이브러리 import
import dart_fss as dart_fss

api_key = '35a9b92886a4bd458dc7be8be0730e05a84ac1e5'
dart_fss.set_api_key(api_key=api_key)

'35a9b92886a4bd458dc7be8be0730e05a84ac1e5'

In [11]:
# dart api를 이용 사업보고서 가져오기
all = dart_fss.api.filings.get_corp_code()
all[0]

df = pd.DataFrame(all)
# 전체 기업중 상장사만 선택
all_listed_corps = df[df['stock_code'].notnull()]
all_listed_corps

,corp_code,corp_name,corp_eng_name,stock_code,modify_date
1946,00260985,한빛네트,"HanbitNet, Inc.",036720,20170630
1958,00264529,엔플렉스,"Nplex,Inc.",040130,20170630
1959,00358545,동서정보기술,"Dong Seo Information Technology Co., Ltd.",055000,20170630
2699,00231567,애드모바일,None,032600,20170630
3771,00359614,리더컴,"LEADER COMM.Co.,Ltd",056140,20170630
...,...,...,...,...,...
112873,01113499,아이티센피엔에스,"ITCENPNS CO.,LTD.",232830,20250509
112874,00488244,원일티엔아이,"WONILTNI CO., LTD.",136150,20250509
112875,01626391,오가노이드사이언스,ORGANOIDSCIENCES LTD.,476040,20250509
112879,00532855,한국유니온제약,"UNION KOREA PHARM CO.,Ltd",080720,20250630


In [13]:
corp_df = pd.DataFrame.from_dict(all)
corp_df = corp_df.dropna(subset = 'stock_code').sort_values('modify_date',ascending=False).reset_index(drop=True)
corp_df['done_YN'] = "N"
corp_df

,corp_code,corp_name,corp_eng_name,stock_code,modify_date,done_YN
0,00959229,세종메디칼,"SEJONG MEDICAL Co.Ltd.,",258830,20250718,N
1,01409095,비트맥스,"BITMAX CO., LTD",377030,20250717,N
2,00288495,에이치씨홈센타,"HC HomeCenter Co.,Ltd",060560,20250717,N
3,00306870,손오공,"SONOKONGCO.,LTD.",066910,20250717,N
4,00568461,에이치씨보광산업,"HC BoKwang Industry Co.,Ltd",225530,20250717,N
...,...,...,...,...,...,...
3872,00260639,대흥멀티미디어통신,daeheung multimedia communication corp,037250,20170630,N
3873,00153852,텔슨정보통신,TelsonInformation&Communication,018180,20170630,N
3874,00248956,비이티,"BET CO., LTD.",036820,20170630,N
3875,00390028,코오롱인터내셔널,KOLON INTERNATIONAL CORP.,063510,20170630,N


In [14]:
corp_code = '00126380' # 삼성전자의 기업번호
url_json = "https://opendart.fss.or.kr/api/list.json"

params = {
    'crtfc_key' : api_key,
    'corp_code' : corp_code ,
    'pblntf_ty' : 'A',
    'bgn_de' : '20230101' ## 사업보고서 시작일!
}

response = requests.get(url_json, params = params)
res = response.json()
if res ['status'] =='013' :
    print(f'{i},{corp_code}, {corp_df.loc[i,"corp_name"]}, 사업보고서개수 : 0')

df_imsi = pd.DataFrame(res['list'])
df_imsi

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,분기보고서 (2025.03),20250515001922,삼성전자,20250515,
1,00126380,삼성전자,005930,Y,사업보고서 (2024.12),20250311001085,삼성전자,20250311,연
2,00126380,삼성전자,005930,Y,분기보고서 (2024.09),20241114002642,삼성전자,20241114,
3,00126380,삼성전자,005930,Y,반기보고서 (2024.06),20240814003284,삼성전자,20240814,
4,00126380,삼성전자,005930,Y,분기보고서 (2024.03),20240516001421,삼성전자,20240516,
5,00126380,삼성전자,005930,Y,사업보고서 (2023.12),20240312000736,삼성전자,20240312,연
6,00126380,삼성전자,005930,Y,분기보고서 (2023.09),20231114002109,삼성전자,20231114,
7,00126380,삼성전자,005930,Y,반기보고서 (2023.06),20230814002534,삼성전자,20230814,
8,00126380,삼성전자,005930,Y,분기보고서 (2023.03),20230515002335,삼성전자,20230515,
9,00126380,삼성전자,005930,Y,사업보고서 (2022.12),20230307000542,삼성전자,20230307,연


In [15]:
  ## API로 데이터 가져오기
    url = 'https://opendart.fss.or.kr/api/document.xml'
    params = {
        'crtfc_key': api_key,
        'rcept_no': '20230307000542',
    }

    r = requests.get(url, params=params)

	## 가져온 데이터를 XML 기반 파싱
    try:
        tree = ET.XML(r.text)
        status = tree.find('status').text
        message = tree.find('message').text
        if (status != '000') & (status != '014') :
            raise ValueError({'status': status, 'message': message})



    except ET.ParseError as e:
        pass

    zf = zipfile.ZipFile(io.BytesIO(r.content))
    info_list = zf.infolist()
    fnames = sorted([info.filename for info in info_list])
    ## 결과물을 리스트에 넣기
    xml_text_list = []
    for fname in fnames:
        xml_data = zf.read(fname)
        try:
            xml_text = xml_data.decode('euc-kr')
        except UnicodeDecodeError as e:
            xml_text = xml_data.decode('utf-8')
        except UnicodeDecodeError as e:
            xml_text = xml_data
        xml_text_list.append(xml_text)

IndentationError: unexpected indent (ipython-input-15-554848998.py, line 2)

In [17]:
from bs4 import BeautifulSoup
import zipfile
import io
import requests
import xml.etree.ElementTree as ET

# API KEY와 접수번호는 이미 확보한 상태라고 가정
params = {
    'crtfc_key': api_key,
    'rcept_no': '20230307000542',
}
r = requests.get('https://opendart.fss.or.kr/api/document.xml', params=params)

def extract_section(soup, keywords=['사업의 내용', '연구개발', '요약 재무정보']):
    results = {}
    for keyword in keywords:
        section = soup.find(lambda tag: tag.name in ['p', 'div', 'span'] and keyword in tag.get_text())
        if section:
            next_content = ""
            current = section.find_next_sibling()
            while current and (not any(kw in current.text for kw in keywords)):
                next_content += current.get_text(separator=' ', strip=True) + '\n'
                current = current.find_next_sibling()
            results[keyword] = next_content.strip()
        else:
            results[keyword] = "(해당 항목 없음)"
    return results


# 압축 해제
zf = zipfile.ZipFile(io.BytesIO(r.content))
xml_text_list = []
for fname in zf.namelist():
    xml_data = zf.read(fname)
    try:
        xml_text = xml_data.decode('euc-kr')
    except:
        try:
            xml_text = xml_data.decode('utf-8')
        except:
            xml_text = xml_data.decode(errors='ignore')
    xml_text_list.append(xml_text)

# HTML 파싱 및 내용 추출
for xml_text in xml_text_list:
    if '사업의 내용' in xml_text:  # 이걸로 본문 판단
        soup = BeautifulSoup(xml_text, 'html.parser')
        break

# 핵심 항목만 추출
extracted = extract_section(soup)
for k, v in extracted.items():
    print(f"\n🔹 {k}\n{v[:500]}...\n")


/tmp/ipython-input-17-3063621860.py:47: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(xml_text, 'html.parser')



🔹 사업의 내용
당사는 본사를 거점으로 한국과 DX 부문 산하 해외 9개 지역총괄, DS 부문 산하 해외 5개 지역총괄의 생산ㆍ판매법인 및 SDC, Harman 산하 종속기업 등 232개의 종속기업으로 구성된 글로벌 전자 기업입니다.각 부문별 주요 제품은 다음과 같습니다.
부문 주 요  제 품 DX 부문 TV, 모니터, 냉장고, 세탁기, 에어컨,HHP, 네트워크시스템, 컴퓨터 등 DS 부문 DRAM, NAND Flash, 모바일AP 등 SDC 스마트폰용 OLED 패널 등 Harman 디지털 콕핏(Digital Cockpit), 텔레매틱스(Telematics), 스피커 등
※ 당사는 2021년 12월 조직개편을 통해 CE 부문과 IM 부문을 DX 부문으로 통합하였으며,    내부 조직체계에 맞추어 SDC를 DS 부문과 별도로 구분하였습니다.

[DX 부문]
DX 부문은 TV, 모니터, 냉장고, 세탁기, 에어컨 및 휴대폰, 태블릿, 웨어러블(Wearable) 제품 등을 생산ㆍ판매하는 사업을 영위하고 있...


🔹 연구개발
...


🔹 요약 재무정보
(1) 당기
(단위 : 백만원)
기업명(*1) 자산 부채 매출액 당기순이익(손실) 삼성디스플레이㈜ 57,302,567 7,282,718 30,779,405 4,365,588 Samsung Electronics America, Inc. (SEA) 37,883,156 12,258,315 46,738,920 219,670 Samsung Asia Pte. Ltd. (SAPL) 26,894,611 2,678,285 - 8,699,679 Harman과 그 종속기업(*2) 17,102,324 6,380,456 13,211,151 631,019 Samsung (China) Semiconductor Co., Ltd. (SCS) 17,095,000 2,970,835 9,679,757 638,385 Samsung Electronics Vietnam THAINGUYEN Co., Ltd. (SEVT) 15,718,299 2,358,140